In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from keras import layers
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import keras
from keras import layers
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('NI.csv', sep=",")

In [3]:
data['date'] = pd.to_datetime(data['date'])

In [4]:
data.sort_values(by='date', ascending=True, inplace=True)

In [5]:
data.drop(columns=['areaName', 'areaCode'], inplace=True)

In [6]:
data.head()

,date,newCasesByPublishDate,cumCasesByPublishDate,newDeaths28DaysByDeathDate,cumDeaths28DaysByDeathDate,newAntibodyTestsByPublishDate,newLFDTestsBySpecimenDate,newPCRTestsByPublishDate,newPCRTestsByPublishDateRollingSum,hospitalCases,newAdmissions,newPeopleVaccinatedCompleteByVaccinationDate,newVaccinesGivenByPublishDate,VaccineRegisterPopulationByVaccinationDate
739,2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN
708,2020-01-04,103.0,689.0,4.0,45.0,NaN,0.0,NaN,NaN,280.0,45.0,NaN,NaN,NaN
678,2020-01-05,87.0,3623.0,12.0,387.0,NaN,0.0,NaN,NaN,269.0,19.0,NaN,NaN,NaN
647,2020-01-06,12.0,4728.0,1.0,532.0,NaN,0.0,NaN,NaN,127.0,3.0,NaN,NaN,NaN
617,2020-01-07,1.0,5761.0,1.0,552.0,NaN,0.0,NaN,NaN,32.0,0.0,NaN,NaN,NaN


In [7]:
data = data.set_index('date')

In [8]:
data_new_cases = data[['newCasesByPublishDate']]

In [9]:
data_new_cases.loc[:,'minus_7'] = data_new_cases.loc[:,'newCasesByPublishDate'].shift(periods=-7)

In [10]:
data_new_cases.loc[:,'minus_14'] = data_new_cases.loc[:,'newCasesByPublishDate'].shift(periods=-14)

In [11]:
data_new_cases.loc[:,'minus_21'] = data_new_cases.loc[:,'newCasesByPublishDate'].shift(periods=-21)

In [12]:
data_new_cases.loc[:,'minus_28'] = data_new_cases.loc[:,'newCasesByPublishDate'].shift(periods=-28)

In [13]:
data_new_cases = data_new_cases.dropna()

In [14]:
data_new_cases.head()

,newCasesByPublishDate,minus_7,minus_14,minus_21,minus_28
date,,,,,
2020-01-04,103.0,685.0,0.0,78.0,456.0
2020-01-05,87.0,391.0,71.0,8.0,10.0
2020-01-07,1.0,85.0,493.0,0.0,11.0
2020-01-08,0.0,66.0,416.0,77.0,7.0
2020-01-10,259.0,7.0,130.0,570.0,6.0


In [15]:
data_new_cases.shape

(697, 5)

In [16]:
n = len(data_new_cases)

In [17]:
target = data_new_cases['newCasesByPublishDate']

In [18]:
data_new_cases.columns

Index(['newCasesByPublishDate', 'minus_7', 'minus_14', 'minus_21', 'minus_28'], dtype='object')

In [19]:
variables = data_new_cases[['minus_7', 'minus_14', 'minus_21', 'minus_28']]

In [20]:
X_train = data_new_cases.iloc[0:int(n*0.7)].drop(['newCasesByPublishDate'], axis=1)
y_train = data_new_cases.iloc[0:int(n*0.7)].drop(columns=variables, axis=1)

In [21]:
X_test = data_new_cases.iloc[int(n*0.9):].drop(['newCasesByPublishDate'], axis=1)
y_test = data_new_cases.iloc[int(n*0.9):].drop(columns=variables, axis=1)

In [22]:
X_val = data_new_cases.iloc[int(n*0.7):int(n*0.9)].drop(['newCasesByPublishDate'], axis=1)
y_val = data_new_cases.iloc[int(n*0.7):int(n*0.9)].drop(columns=variables, axis=1)

In [23]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,1)).fit(X_train)

In [24]:
X_train_scaled = scaler.transform(X_train)

In [25]:
model = keras.Sequential()
model.add(layers.LSTM(500, return_sequences=True))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))


In [27]:
model.compile(loss=tf.losses.MeanSquaredError(),
            optimizer=tf.optimizers.Adam(),
            metrics=[tf.metrics.MeanAbsoluteError(),])

In [31]:
model.fit(X_train_scaled, y_train, epochs=500, batch_size=32)

Epoch 1/500


ValueError: in user code:

    File "/Users/simonmclain/miniforge3/envs/tf24/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Users/simonmclain/miniforge3/envs/tf24/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/simonmclain/miniforge3/envs/tf24/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Users/simonmclain/miniforge3/envs/tf24/lib/python3.8/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/Users/simonmclain/miniforge3/envs/tf24/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/simonmclain/miniforge3/envs/tf24

In [ ]:
results = []
names = []
for name, mod in model:
    tsvc = TimeSeriesSplit(n_splits=2)
    cv_results = cross_val_score(model, X_train, y_train, cv=tsvc, scoring='r2')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
model = MLPRegressor()

In [ ]:
param_search = {
    "hidden_layer_sizes": [(1,),(50,),(100,),(150,),(200,)],
    "activation": ["identity", "logistic", "tanh", "relu"],
    "solver": ["lbfgs", "sgd", "adam"],
    "alpha": [0.00005,0.0005],
    "learning_rate": ['constant', 'invscaling', 'adaptive']
}

In [ ]:
tsvc = TimeSeriesSplit(n_splits=2)
gsearch = GridSearchCV(estimator=model, cv=tsvc, param_grid=param_search, scoring='r2')
gsearch.fit(X_train, y_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

In [ ]:
print(best_model)

In [ ]:
print(best_score)

In [ ]:
from sklearn.metrics import r2_score

y_pred = best_model.predict(X_test)

print(r2_score(y_test, y_pred))